In [1]:
import sys
sys.path.append("../")

In [2]:
import os

import numpy as np
import pandas as pd
import random

from agents.matchmanager import buildMatchManager
from core.game.state import vectorStateInfo, vectorState

from datetime import datetime
from multiprocessing import Pool

In [3]:
agents = [
    'RandomAgent', 
#     'AlphaBetaAgent', 
    'GreedyAgent'
]
scenarios = [
#     'scenarioJunctionExo', 
    'scenarioJunction', 
#     'scenarioRoadblock', 
#     'scenarioBridgeHead', 
#     'scenarioCrossingTheCity'
]

In [4]:
def game(args):
    scenario, red, blue, seed = args
    vectors=[]

    output = f'../data/{scenario}.{red}.{blue}.{seed}.pkl.gz'
    
    if os.path.exists(output):
        return
    
    mm = buildMatchManager('', scenario, red, blue, seed=seed)
    
    while not mm.end:
        mm.nextStep()

    cols = vectorStateInfo()
    data = [vectorState(x) for x in mm.states_history]

    df = pd.DataFrame(columns=cols, data=data)
    
    df['winner'] = mm.winner
    df['meta_p_red'] = red
    df['meta_p_blue'] = blue
    df['meta_seed'] = seed
    df['meta_scenario'] = scenario
    
    df.to_pickle(output, compression='gzip')


In [5]:
args = []

np.random.seed(20210215)

for _ in range(128):
    scenario = np.random.choice(scenarios)
    red = np.random.choice(agents)
    blue = np.random.choice(agents)
    seed = np.random.randint(100000000, 999999999)
    
    args.append((scenario, red, blue, seed))

In [6]:
with Pool(64) as p:
    p.map(game, args)

Compress everything in a single file, for each scenario

In [7]:
def compress(scenario):
    today = datetime.today().strftime('%Y-%m-%d')
    data_dir = '../data'
    
    files = [f for f in os.listdir(data_dir) if scenario in f]
    
    dfs = [
        pd.concat([
            pd.read_pickle(os.path.join(data_dir, f), compression='gzip') for f in files[i:i+1000]
        ]) for i in range(0, len(files), 1000)
    ]
    
    dfs = pd.concat(dfs)
    
    dfs.to_pickle(f'../data.{today}.{scenario}.pkl.gz', compression='gzip')

In [8]:
with Pool(len(scenarios)) as p:
    p.map(compress, scenarios)